In [1]:
sentences1 = ["私は公園で遊びます。", "彼女は本を読みます。", "彼はテニスをします。", "私たちは映画を見に行きます。", "私は日本語を勉強しています。"]

sentences2 = ["私は公園で遊びます。", "私は公園で楽しみます。", "私は公園で転びます。", "私は公園で騒ぎます", "私は公園で歩きます。"]


In [2]:
sentences1 = ["私は公園で遊びます。 彼女は本を読みます。", "彼はテニスをします。 私たちは映画を見に行きます。", "私は公園で騒ぎます。 私は日本語を勉強しています。"]

sentences2 = ["私は公園で遊びます。 私は公園で楽しみます。", "私は公園で転びます。 私は公園で騒ぎます", "私は公園で転びます。 私は公園で歩きます。"]

- pre-trained BERT
- word2vec
- doc2vec
- sentence BERT

In [3]:
# pre-trained BERT

# import numpy as np
# import torch
# from transformers import BertJapaneseTokenizer, BertModel

# BERT_MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'
# tokenizer = BertJapaneseTokenizer.from_pretrained(BERT_MODEL_NAME)
# model_bert = BertModel.from_pretrained(BERT_MODEL_NAME)

# def sentence_to_vector_bert(model_bert, tokenizer, sentence):
#     tokens = tokenizer(sentence, add_special_tokens=True)["input_ids"]
#     input = torch.tensor(tokens).unsqueeze(0)
#     with torch.no_grad():
#         outputs = model_bert(input, output_hidden_states=True)
#         last_hidden_state = outputs[0][:, 0, :]
#         averaged_hidden_state = last_hidden_state.mean(
#             dim=0).unsqueeze(0).reshape(-1)
#     return averaged_hidden_state


In [4]:
# sentence_to_vector_bert(model_bert, tokenizer, sentences1[0])

In [5]:
# word2vec

import re
import csv
import numpy as np
import MeCab
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity

model_dir = '/Users/iomacbookair2/Documents/lab/DEIM2023/entity_vector/entity_vector.model.bin'
model_word2vec = KeyedVectors.load_word2vec_format(model_dir, binary=True)

# http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/


In [24]:
def calc_similarity_word2vec(sentence1, sentence2, model):
  mecab = MeCab.Tagger()

  pre_sentence1 = sentence1.split(" ")
  sentence1_words = []
  for s1 in pre_sentence1:
    sentence1_words += [line.split("\t")[0]
                        for line in mecab.parse(s1).split("\n")[:-2]]
  print(f"sentence1_words: {sentence1_words}")

  pre_sentence2 = sentence2.split(" ")
  sentence2_words = []
  for s2 in pre_sentence2:
    sentence2_words += [line.split("\t")[0]
                        for line in mecab.parse(s2).split("\n")[:-2]]
  print(f"sentence2_words: {type(sentence2_words)}")

  if not sentence1_words or not sentence2_words:
    similarity = 0
  elif (len(sentence1_words) == 1 and len(sentence2_words) != 1) or (len(sentence2_words) == 1 and len(sentence1_words) != 1):
    similarity = 0
  elif all(re.match(r'.*[a-zA-Z].*', word) for word in sentence1_words) or all(re.match(r'.*[a-zA-Z].*', word) for word in sentence2_words):
    similarity = 0
  else:
    # Compute word embeddings for each sentence
    sentence1_embedding = np.mean([model[word]
                                  for word in sentence1_words if word in model], axis=0)
    print(f"sentence1_embedding: {sentence1_embedding}")
    sentence2_embedding = np.mean([model[word]
                                  for word in sentence2_words if word in model], axis=0)
    print(f"sentence2_embedding: {sentence2_embedding}")
    if np.isnan(sentence1_embedding).any() or np.isnan(sentence2_embedding).any():
        similarity = 0
    else:
      similarity = cosine_similarity([sentence1_embedding], [sentence2_embedding])[0][0]

  print(f"similarity: {similarity}")

  return similarity


def calc_average_similarity_word2vec(sentences):
    similarities = []
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            similarity = calc_similarity_word2vec(
                sentences[i], sentences[j], model_word2vec)
            similarities.append(similarity)

    q = [0, 0.25, 0.5, 0.75, 1]
    outputs = {}
    for i in range(len(q)):
        outputs[f"quantile_{i}"] = np.quantile(similarities, q[i])
    outputs["average_similarity"] = np.mean(similarities)
    outputs["size"] = len(list(np.sort(np.array(similarities))))
    outputs["width_q"] = outputs["quantile_3"] - outputs["quantile_1"]

    return outputs


In [18]:
sentences1 = ["うす", "うっす"]


In [25]:
calc_average_similarity_word2vec(sentences1)

sentence1_words: ['うす']
sentence2_words: <class 'list'>
sentence1_embedding: [ 9.33173373e-02  1.32539362e-01  7.16414079e-02 -3.78423870e-01
  2.13147979e-02 -8.52847844e-02 -1.76157489e-01 -1.83672637e-01
 -1.09914519e-01  1.43518209e-01 -1.52750361e+00 -4.91113484e-01
 -5.72648704e-01 -6.41453087e-01  5.52469432e-01  1.77199021e-01
  9.94765282e-01  2.68994302e-01 -4.34988618e-01  4.35320735e-01
  1.81212962e-01 -1.02123010e+00  5.73091209e-01 -9.29585814e-01
  4.00175840e-01 -3.53511781e-01  1.29680768e-01  1.21582091e-01
 -4.53335792e-01 -3.31579834e-01  8.68348598e-01  3.19731176e-01
  2.55779684e-01  4.44218457e-01  5.84572673e-01  2.53961813e-02
 -1.69237897e-01 -8.05172980e-01  6.74818754e-01  5.03557920e-01
  5.65926433e-01 -8.10618885e-03  5.07991731e-01 -6.81066573e-01
  8.67584571e-02  2.55616039e-01 -4.87629652e-01  6.55323491e-02
 -5.60740113e-01  6.30179718e-02 -2.37908125e-01 -4.35338527e-01
  1.25867069e+00 -8.06568265e-02 -1.52061343e-01  5.45516431e-01
 -3.90336484e

{'quantile_0': 0,
 'quantile_1': 0.0,
 'quantile_2': 0.0,
 'quantile_3': 0.0,
 'quantile_4': 0,
 'average_similarity': 0.0,
 'size': 1,
 'width_q': 0.0}

In [ ]:
calc_average_similarity_word2vec(sentences2)